<a href="https://colab.research.google.com/github/CristhianSeverino/Data_Engeerig/blob/En-proceso/Practica_SQL_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook de Practica de SQL**

> Este Notebook es creacion de: **Cristhian Calle Severino**.

**Si este Notebook te fue de ayuda, me encantaria saberlo**
**Diviertete Creando**

*   **Github**: https://github.com/CristhianSeverino
*   **Linkedin**: https://www.linkedin.com/in/cristhianandrescalleseverino/








>Este proyecto **Incluye:**😎

* Creacion de datos sinteticos. Se crean cinco tablas sinteticas usando faker. estas tablas obedecen a un diagrama de estrella.
* Exploracion y coneccion con una db. usando metodos magicos del notebook.
* Incluye Resopuesta correcta y Prueba.


# **Instalacción E Importación de Librerias**

In [23]:
!pip install faker
!pip install pandas sqlalchemy mysqlclient faker
!pip install ipython-sql==0.5.0 prettytable==3.9.0

print("="*150)
print(" "*50+"Librerias Instaladas ;)")
print("="*150)
from faker import Faker
import pandas as pd
from datetime import datetime
import random
from sqlalchemy import create_engine
import numpy as np
import os as os
import locale


print("="*150)
print(" "*50+"Librerias Importadas ;)")
print("="*150)

%load_ext sql
print("="*150)
print(" "*50+"Extensión sql Cargada ;)")
print("="*150)

                                                  Librerias Instaladas ;)
                                                  Librerias Importadas ;)
The sql extension is already loaded. To reload it, use:
  %reload_ext sql
                                                  Extensión sql Cargada ;)


# **Creación de Datos Sinteticos**

  * El presente notebook y sus ejercicios de SQL se basan en una base de datos creada con **datos sinteticos**. en este apartado encontraras las **funciones y la ejecucion de las funciones** que lo hacen posible.
  * Adaptalas acorde a lo que necesites o quieras crear.**Puedes agrandar los datasets, a tu gusto 😎**.
  * La Base de datos se carga en esta seccion a **SQLite** ideal para practicar SQL en entorno Colab.

**Diviertete Explorando este Proyecto y creando con el. Prospero día ☕**

In [24]:
#=================================   Generar Diagrama Estrella   =============================================
# Modifica num_filas_hechos segun necesidad
def generar_esquema_estrella(num_filas_hechos=50000):
    fake = Faker('es_CO')

    #=======================   Tablas de Dimensiones   ===============

    # 1 Dimensión cliente: Simula datos base del cliente
    dim_cliente_data = []
    ids_usados = set()
    num_clientes = 10000 # Definimos el número de clientes
    while len(dim_cliente_data) < num_clientes: # Usamos num_clientes en el bucle
        id_candidato = fake.random_int(min=1000000000, max=1000100000)
        if id_candidato not in ids_usados:
            ids_usados.add(id_candidato)
            dim_cliente_data.append({
                'id_cliente': id_candidato,
                'nombre': fake.name(),
                'categoria': fake.random_element(['BtoB', 'BtoC']),
                'correo': fake.email(),
                'fecha_nacimiento': fake.date_of_birth(minimum_age=18, maximum_age=45),
                'telefono': fake.phone_number(),
            })
    dim_cliente = pd.DataFrame(dim_cliente_data)

    # 2 Dimensión Producto: Simula diferentes productos.
    num_productos = 350
    dim_producto_data = []
    for i in range(num_productos):
        dim_producto_data.append({
            'id_producto': i,
            'nombre_producto': fake.catch_phrase(),
            'categoria': fake.random_element(['Electrónica', 'Ropa', 'Juguetes']),
            'precio_unitario': round(random.uniform(5.0, 500.0), 2),
            'stock_code': f"PROD{i+1:04d}"
        })
    dim_producto = pd.DataFrame(dim_producto_data)

    # 3 Dimensión Tiempo: Registra la fecha de las Transacciones.
    fechas = pd.to_datetime(pd.date_range('2024-01-01', '2024-12-31'))
    dim_tiempo_data = []
    for i, fecha in enumerate(fechas):
        dim_tiempo_data.append({
            'id_fecha': i,
            'fecha': fecha,
            'año': fecha.year,
            'mes': fecha.month,
            'dia': fecha.day,
            'dia_semana': fecha.day_name(),
        })
    dim_tiempo = pd.DataFrame(dim_tiempo_data)

    # 4. Dimensión Ubicación: Desgloza la dirección
    municipios = ['Manizales', 'Bogota', 'Pereira', 'Medellin', 'Cartagena', 'Barranquilla', 'Santa Marta', 'Neiva', 'Mitu', 'Valledupar', 'Mocoa']
    num_ubicaciones = len(municipios)
    dim_ubicacion_data = []
    for i, municipio in enumerate(municipios):
        dim_ubicacion_data.append({
            'id_ubicacion': i,
            'nombre_municipio': municipio,
            'direccion': fake.address(),
        })
    dim_ubicacion = pd.DataFrame(dim_ubicacion_data)

    # 5 Tabla de hechos (Ventas)
    fact_ventas_data = {
        'id_venta': np.arange(num_filas_hechos),
        # Claves foráneas de las Dimensiones
        'id_cliente': np.random.randint(0, num_clientes, num_filas_hechos),
        'id_producto': np.random.randint(0, num_productos, num_filas_hechos),
        'id_fecha': np.random.randint(0, len(fechas), num_filas_hechos),
        'id_ubicacion': np.random.randint(0, num_ubicaciones, num_filas_hechos),
        # Métricas de Negocio
        'cantidad_vendida': np.random.randint(1, 10, num_filas_hechos),
    }
    fact_ventas = pd.DataFrame(fact_ventas_data)

    # Calcular el ingreso total basándose en los precios de los productos.
    fact_ventas = fact_ventas.merge(
        dim_producto[['id_producto', 'precio_unitario']],
        on='id_producto',
        how='left'
    )
    fact_ventas['ingreso_total'] = fact_ventas['precio_unitario'] * fact_ventas['cantidad_vendida']
    fact_ventas = fact_ventas.drop(columns=['precio_unitario'])

    return {
        'fact_ventas': fact_ventas,
        'dim_cliente': dim_cliente,
        'dim_producto': dim_producto,
        'dim_tiempo': dim_tiempo,
        'dim_ubicacion': dim_ubicacion
    }


In [25]:
#=================================   Generar base de Datos SQL   =============================================
# Recuerda esta funcion depende en gran parte de la estructura creada en la función anterior.
# Antes de modificar valida que tenga coheencia con la funcion anterior y la celda de ejecución ;)
def cargar_a_sqlite(dataframes_dict, db_path='esquema_ventas_negocio.db'):
    """
    Carga El Diccionario de DataFrames de pandas a una base de datos SQLite.
    La base de datos se guarda en un archivo local.

    Args:
        dataframes_dict (dict): Un diccionario con los DataFrames a cargar.
        db_path (str): La ruta del archivo donde se guardará la base de datos SQLite.
    """
    try:
        # Cadena de conexión a un archivo SQLite local
        engine_string = f'sqlite:///{db_path}'
        engine = create_engine(engine_string)

        # Si el archivo ya existe, lo eliminamos para recrearlo limpio en cada ejecución
        if os.path.exists(db_path):
            os.remove(db_path)
            print(f"🔄 Archivo de base de datos '{db_path}' existente eliminado.")

        print(f"🔗 Conexión a la base de datos '{db_path}' establecida.")

        # Cargar cada DataFrame en su respectiva tabla
        for table_name, df in dataframes_dict.items():
            print(f"⏳ Cargando tabla '{table_name}'...")

            # 'if_exists'='replace' crea la tabla si no existe o la reemplaza si ya lo hace
            df.to_sql(
                name=table_name,
                con=engine,
                if_exists='replace',
                index=False,
            )
            print("="*150)
            print(f"{' '} ✅ Tabla '{table_name}' cargada exitosamente.")
            print("="*150)

        print("\n🎉 ¡Todos los datos han sido cargados a SQLite!")

    except Exception as e:
        print(f"❌ Ocurrió un error al cargar los datos: {e}")


**Ejecutar Funciones**

In [26]:
#=======================================   Ejecución de Funciones    =====================================

# 1. Generar los DataFrames con la función `generar_esquema_estrella`

print("Iniciando la generación de los datos de negocio... 📈")

# Puedes ajustar el número de filas de la tabla de hechos aquí
tablas_negocio = generar_esquema_estrella(num_filas_hechos=100000)

print("Datos generados exitosamente. 🎉")

# 2. Cargar los DataFrames en la base de datos SQLite con `cargar_a_sqlite`

print("Iniciando la carga de datos en la base de datos SQLite... 💾")
# La función creará un archivo llamado 'esquema_ventas_negocio.db'
cargar_a_sqlite(tablas_negocio)

print("Carga completa. ✅")

Iniciando la generación de los datos de negocio... 📈
Datos generados exitosamente. 🎉
Iniciando la carga de datos en la base de datos SQLite... 💾
🔄 Archivo de base de datos 'esquema_ventas_negocio.db' existente eliminado.
🔗 Conexión a la base de datos 'esquema_ventas_negocio.db' establecida.
⏳ Cargando tabla 'fact_ventas'...
  ✅ Tabla 'fact_ventas' cargada exitosamente.
⏳ Cargando tabla 'dim_cliente'...
  ✅ Tabla 'dim_cliente' cargada exitosamente.
⏳ Cargando tabla 'dim_producto'...
  ✅ Tabla 'dim_producto' cargada exitosamente.
⏳ Cargando tabla 'dim_tiempo'...
  ✅ Tabla 'dim_tiempo' cargada exitosamente.
⏳ Cargando tabla 'dim_ubicacion'...
  ✅ Tabla 'dim_ubicacion' cargada exitosamente.

🎉 ¡Todos los datos han sido cargados a SQLite!
Carga completa. ✅


# **Ejercicios SQL**

**Despliega**
> Aqui encontraras 10 ejercicios basicos de SQL. sientete libre de añadir celdas y crear consultas mas complejas. **Hazte las Siguientes Preguntas:**

* ¿Si fuera el Director Ejecutivo que desearia ver. que Indicadores aporta valor y me ahorrarian tiempo?.
* ¿Si fuera el Analista de datos senior. Que informacion analizaria en busca de insights de alto valor?.
* ¿Que indicadores debo consultar dia a dia, cuales semanalmente y cuales trimestralmente?




In [27]:
%load_ext sql
%sql sqlite:///esquema_ventas_negocio.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


1. **Ver solamente 10 facturas de ventas:** Esta consulta te permite ver las primeras 10 filas de la tabla fact_ventas, que contiene las métricas del negocio.

In [28]:
%%sql SELECT*FROM fact_ventas LIMIT 10;

 * sqlite:///esquema_ventas_negocio.db
Done.


id_venta,id_cliente,id_producto,id_fecha,id_ubicacion,cantidad_vendida,ingreso_total
0,1808,229,2,8,5,2152.0
1,1103,212,326,2,6,604.86
2,326,64,259,3,4,135.0
3,7482,50,156,2,4,967.56
4,667,305,248,6,1,498.86
5,1070,303,122,7,3,107.39999999999999
6,8762,153,148,5,4,1702.96
7,156,30,312,5,6,517.74
8,4934,289,211,2,3,576.48
9,1031,159,14,5,9,3132.09



>%%sql
SELECT *
FROM fact_ventas
LIMIT 10;



---
2. **Contar el número total de ventas:** Con esta consulta, puedes saber cuántas transacciones se registraron en total en la tabla de hechos.


In [29]:
%%sql SELECT COUNT(*) AS total_ventas FROM fact_ventas;

 * sqlite:///esquema_ventas_negocio.db
Done.


total_ventas
100000


>%%sql
SELECT COUNT(*) AS total_ventas
FROM fact_ventas;



---
3. **Calcular el ingreso total:** Esta consulta suma todos los valores de la columna ingreso_total para obtener el total de ingresos generados.


In [30]:
%%sql SELECT SUM(ingreso_total) AS ingreso_neto_total FROM fact_ventas;

 * sqlite:///esquema_ventas_negocio.db
Done.


ingreso_neto_total
131952990.0300019





>%%sql
SELECT SUM(ingreso_total) AS ingreso_neto_total
FROM fact_ventas;




---
4. **Encontrar las 5 ventas con mayor ingreso:** Con ORDER BY y DESC, puedes ordenar los resultados de mayor a menor para encontrar las transacciones más grandes.


In [31]:
%%sql
SELECT *
FROM fact_ventas
ORDER BY ingreso_total DESC
LIMIT 5;

 * sqlite:///esquema_ventas_negocio.db
Done.


id_venta,id_cliente,id_producto,id_fecha,id_ubicacion,cantidad_vendida,ingreso_total
1677,6987,311,312,4,9,4496.49
2324,8266,311,213,8,9,4496.49
5121,9598,311,255,1,9,4496.49
8685,1124,311,183,3,9,4496.49
8733,800,311,2,0,9,4496.49


> %%sql
SELECT *
FROM fact_ventas
ORDER BY ingreso_total DESC
LIMIT 5;



---
5. **Contar el número de clientes únicos:**


In [32]:
%%sql
SELECT COUNT(DISTINCT id_cliente) AS total_clientes_unicos
FROM dim_cliente;

 * sqlite:///esquema_ventas_negocio.db
Done.


total_clientes_unicos
10000


>%%sql
SELECT COUNT(DISTINCT id_cliente) AS total_clientes_unicos
FROM dim_cliente;



---
6. **Sumar la cantidad vendida por categoría de producto:** Usando GROUP BY, puedes agrupar las ventas por categoria y sumar las cantidades vendidas, uniendo la tabla de hechos con la de dimensiones.


In [33]:
%%sql SELECT T2.categoria, SUM(T1.cantidad_vendida) AS cantidad_total_vendida FROM fact_ventas AS T1 JOIN dim_producto AS T2 ON T1.id_producto = T2.id_producto GROUP BY T2.categoria;

 * sqlite:///esquema_ventas_negocio.db
Done.


categoria,cantidad_total_vendida
Electrónica,166913
Juguetes,175592
Ropa,157087


>%%sql
SELECT
    T2.categoria,
    SUM(T1.cantidad_vendida) AS cantidad_total_vendida
FROM
    fact_ventas AS T1
JOIN
    dim_producto AS T2 ON T1.id_producto = T2.id_producto
GROUP BY
    T2.categoria;



---
7. **Calcular el ingreso promedio por mes:** Esta consulta une la tabla de hechos con la de dimensión de tiempo para calcular los ingresos promedio de cada mes.


In [34]:
%%sql SELECT T2.mes, AVG(T1.ingreso_total) AS ingreso_promedio_mensual FROM fact_ventas AS T1 JOIN dim_tiempo AS T2 ON T1.id_fecha = T2.id_fecha GROUP BY T2.mes ORDER BY T2.mes;

 * sqlite:///esquema_ventas_negocio.db
Done.


mes,ingreso_promedio_mensual
1,1321.509599568396
2,1332.098514056225
3,1311.6001264933157
4,1319.209414887201
5,1313.404796660703
6,1308.858332321885
7,1328.5923847528272
8,1334.0279624536702
9,1318.123867443555
10,1315.8111419347163


>%%sql
SELECT
    T2.mes,
    AVG(T1.ingreso_total) AS ingreso_promedio_mensual
FROM
    fact_ventas AS T1
JOIN
    dim_tiempo AS T2 ON T1.id_fecha = T2.id_fecha
GROUP BY
    T2.mes
ORDER BY
    T2.mes;



---
8. **Encontrar los productos más vendidos:** Con esta consulta, puedes identificar los productos más populares basándote en la cantidad total vendida.


In [35]:
%%sql SELECT T2.nombre_producto, SUM(T1.cantidad_vendida) AS cantidad_total FROM fact_ventas AS T1 JOIN dim_producto AS T2 ON T1.id_producto = T2.id_producto GROUP BY T2.nombre_producto ORDER BY cantidad_total DESC LIMIT 10;

 * sqlite:///esquema_ventas_negocio.db
Done.


nombre_producto,cantidad_total
Ameliorated uniform capability,1693
Reverse-engineered asynchronous access,1662
Managed non-volatile conglomeration,1649
Versatile holistic function,1646
Devolved eco-centric system engine,1641
Fully-configurable bi-directional analyzer,1629
Devolved stable collaboration,1626
Persistent solution-oriented installation,1620
Persistent background hub,1616
Diverse upward-trending alliance,1616


>%%sql
SELECT
    T2.nombre_producto,
    SUM(T1.cantidad_vendida) AS cantidad_total
FROM
    fact_ventas AS T1
JOIN
    dim_producto AS T2 ON T1.id_producto = T2.id_producto
GROUP BY
    T2.nombre_producto
ORDER BY
    cantidad_total DESC
LIMIT 10;



---
9. **Filtrar ventas de una categoría específica:** Esta consulta utiliza la cláusula WHERE para filtrar todas las ventas que pertenecen a la categoría 'Electrónica'.



In [36]:
%%sql
SELECT
    T1.*
FROM
    fact_ventas AS T1
JOIN
    dim_producto AS T2 ON T1.id_producto = T2.id_producto
WHERE
    T2.categoria = 'Electrónica'
LIMIT 10;

 * sqlite:///esquema_ventas_negocio.db
Done.


id_venta,id_cliente,id_producto,id_fecha,id_ubicacion,cantidad_vendida,ingreso_total
897,1103,9,222,0,8,3355.2
999,974,9,203,6,7,2935.7999999999997
1110,6838,9,124,10,7,2935.7999999999997
2480,8112,9,147,10,1,419.4
2967,107,9,346,0,8,3355.2
3332,6248,9,330,2,6,2516.3999999999996
3523,7419,9,109,0,4,1677.6
3637,7460,9,42,1,5,2097.0
4367,3265,9,20,1,9,3774.6
4756,4515,9,50,8,7,2935.7999999999997


>%%sql
SELECT
    T1.*
FROM
    fact_ventas AS T1
JOIN
    dim_producto AS T2 ON T1.id_producto = T2.id_producto
WHERE
    T2.categoria = 'Electrónica'
LIMIT 10;



---
10. **Contar ventas por día de la semana:** Esta consulta une la tabla de hechos con la de dimensión de tiempo para saber en qué días de la semana se vendió más.


In [37]:
%%sql SELECT T2.dia_semana, COUNT(T1.id_venta) AS numero_de_ventas FROM fact_ventas AS T1 JOIN dim_tiempo AS T2 ON T1.id_fecha = T2.id_fecha GROUP BY T2.dia_semana ORDER BY numero_de_ventas DESC;

 * sqlite:///esquema_ventas_negocio.db
Done.


dia_semana,numero_de_ventas
Tuesday,14683
Monday,14403
Saturday,14326
Friday,14184
Thursday,14164
Sunday,14138
Wednesday,14102




>%%sql
SELECT
    T2.dia_semana,
    COUNT(T1.id_venta) AS numero_de_ventas
FROM
    fact_ventas AS T1
JOIN
    dim_tiempo AS T2 ON T1.id_fecha = T2.id_fecha
GROUP BY
    T2.dia_semana
ORDER BY
    numero_de_ventas DESC;



# **DIVIERTE CREANDO🤗**

*Si este poyecto te fue de ayuda. pasa por mi linkedin y cuentame como te ayudo. ☕*

* **linkedink:** https://www.linkedin.com/in/cristhianandrescalleseverino/

He dejado Las siguientes celdas para que agregues tus consultas personalizadas, animate a **crear**.

***Ser cada día un 1% mejor es la clave 🔥***